In [ ]:
import pennylane as qml
from pennylane import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import random
from itertools import product

np.random.seed(42)

# Parte 1: Automatizar la definición de los grafos.

def Grafo_Manual(caso=1):
    G = nx.Graph()
    
    # Definir aristas y pesos según el caso
    if caso == 2:
        # Caso 2: aristas con pesos específicos
        aristas_con_pesos = {(0, 1): 1.0, (0, 5): 0.3, (1, 6): 0.3, (2, 5): 0.3, (2, 4): 1.0, (3, 4): 0.3, (3, 6): 1.0}
        print("Caso 2")
        print("Aristas y sus pesos")
        print(aristas_con_pesos)
    else:
        # Caso 1: todas las aristas con peso 1
        aristas_con_pesos = {(0, 5): 1.0, (0, 4): 1.0, (1, 3): 1.0, (1, 2): 1.0, (2, 5): 1.0, (3, 4): 1.0}
        print("Caso 1")
        print("Aristas y sus pesos")
        print(aristas_con_pesos)
    # Añadir aristas y pesos al grafo
    for (u, v), peso in aristas_con_pesos.items():
        G.add_edge(u, v, weight=peso)
    
    return G

# Dibujar el grafo con pesos
def dibujar_Grafo(grafo):
    pos = nx.spring_layout(grafo)  

    # Dibujar nodos y etiquetas
    plt.figure(figsize=(8, 6))
    nx.draw(grafo, pos, with_labels=True, node_color='skyblue', node_size=700, font_size=10, font_color='black', font_weight='bold')

    # Añadir los pesos como etiquetas en las aristas
    edge_labels = {(u, v): f"{data['weight']:.1f}" for u, v, data in grafo.edges(data=True)}
    nx.draw_networkx_edge_labels(grafo, pos, edge_labels=edge_labels, font_color='red')

    plt.title("Grafo Automático con Pesos en las Aristas")
    plt.tight_layout()
    plt.savefig('grafo_automatico_con_pesos.png')
    plt.show()


# Parámetros: cantidad de nodos
n_wires = 6 # importante especificar la cantidad de nodos que se utiliza
caso = 1
grafo = Grafo_Manual(caso=caso)
dibujar_Grafo(grafo)


# Parte 2: Definición del Mezclador y Unitario de Control

graph = list(grafo.edges())  # Asegurarse de que las aristas están en forma de lista de tuplas

# Definición del Mezclador
def U_B(beta):
    for wire in range(n_wires):
        qml.RX(2 * beta, wires=wire)

# Definición del Unitario de Control
def U_C(gamma):
    for edge in graph:
        wire1, wire2 = edge
        qml.CNOT(wires=[wire1, wire2])
        qml.RZ(2 * gamma, wires=wire2) 
        qml.CNOT(wires=[wire1, wire2])

# Función auxiliar para convertir bitstring a entero
def bitstring_to_int(bit_string_sample):
    return int("".join(str(int((1 - bs) / 2)) for bs in bit_string_sample), base=2)

# Parte 3: Definición del circuito cuántico

dev = qml.device('lightning.qubit', wires=n_wires, shots=1000)

@qml.qnode(dev)
def circuit(gammas, betas, edge=None, n_layers=1):
    # Inicialización con entrelazamiento  
    qml.Hadamard(wires=0)
    for i in range(n_wires-1):
        qml.CNOT(wires=[i,i+1]) 


    for i in range(n_layers):
        U_C(gammas[i])
        U_B(betas[i])
    if edge is None:
        return [qml.sample(qml.PauliZ(wires=i)) for i in range(n_wires)]
    H = qml.PauliZ(edge[0]) @ qml.PauliZ(edge[1])
    return qml.expval(H)

# Parte 4: Optimización del MAX-CUT con umbral de cambio de costo (threshold)
def contar_cortes(bit_string, grafo):
    """Cuenta el número de cortes dados un bitstring y un grafo"""
    corte = 0
    for (i, j) in grafo.edges():
        if bit_string[i] != bit_string[j]:  # Si están en diferentes subconjuntos
             corte += grafo[i][j]['weight']
    return corte

def qaoa_maxcut(n_layers=1, threshold=0.005, min_steps=50, max_steps=100):
    print("\np={:d}".format(n_layers))

    # Inicializar parámetros
    init_params = 0.01 * np.random.rand(2, n_layers, requires_grad=True)
    costs = []  # Para guardar los costos en cada iteración

    def objective(params):
        gammas = params[0]
        betas = params[1]
        neg_obj = 0
        for edge in graph:
            wire1, wire2 = edge
            weight = grafo[wire1][wire2]['weight']  # Obtener el peso de la arista
            neg_obj -= 0.5 *weight * (1 - circuit(gammas, betas, edge=edge, n_layers=n_layers))  # Consideramos el peso
        return neg_obj

    # Optimización
    opt = qml.AdagradOptimizer(stepsize=0.5)
    params = init_params
    prev_cost = float('inf')  # Inicializamos el costo anterior con infinito
    with open('Tabla.txt', 'w', encoding='utf-8') as T:
        for i in range(max_steps):
            params = opt.step(objective, params)
            cost = -objective(params)  # Guardar el costo actual
            costs.append(cost)
        
            # Verificar el cambio en el costo después de las primeras 50 iteraciones
            if i >= min_steps:
                change_percent = abs(prev_cost - cost) / prev_cost
                if change_percent < threshold:
                    print(f"Se alcanzó la convergencia en el paso {i + 1} con un cambio de {change_percent*100:.2f}%")
                    break
        
            prev_cost = cost

            if (i + 1) % 1 == 0:
                print(f"Objective after step {i + 1:5d}: {cost:.7f}")
           
                T.write(f"Objective after step {i + 1:5d}: {cost:.7f}\n")  

    # Parte 4.4: Contar los resultados
    bit_strings = []
   
   
    samples = circuit(params[0], params[1], edge=None, n_layers=n_layers)
    bit_strings = [bitstring_to_int(sample) for sample in np.array(samples).T] 



    bit_strings = np.array(bit_strings)  # Convertimos a un array de numpy
    counts = np.bincount(bit_strings)
   
    # Encontramos la frecuencia máxima
    max_freq = np.max(counts)

    most_freq_bit_string = np.argmax(counts)
 # Convertir bitstring a lista binaria
    best_bitstring = list(map(int, format(most_freq_bit_string, "0{}b".format(n_wires))))
  # Contar el número de cortes en el bitstring más frecuente
    max_cortes = contar_cortes(best_bitstring, grafo)

    # Encontramos todos los índices (bit strings) con la frecuencia máxima
    most_freq_bit_strings = np.where(counts == max_freq)[0]
    with open('Contenido_Cuantico.txt', 'w', encoding='utf-8') as CC:
        print("Número máximo de cortes: ", max_cortes)

        print("Optimized (gamma, beta) vectors:\n{}".format(params))

    # Imprimimos todos los bit strings que tienen máxima frecuencia
        print("Most frequently sampled bit strings are:")
        for bit_string in most_freq_bit_strings:
            print("{:04b}".format(bit_string))
            CC.write("Bitstring frecuente: {:04b}\n".format(bit_string))

        print("Funcion Objetivo: ", -objective(params))
        CC.write(f"Número máximo de corteos: {max_cortes}\n")
        
        CC.write("Optimized (gamma, beta) vectors:\n{}".format(params))
        
            

    return -objective(params), bit_strings, params, costs

# Parte 5: Búsqueda exhaustiva para MAX-CUT
def exhaustive_search_maxcut(graph):
    best_cost = float('-inf')
    best_bitstrings = []

    for bitstring in product([0, 1], repeat=n_wires):  # El bitstring es una tupla de bits
        cost = 0
        for edge in graph:
            node1, node2 = edge  # Obtener los nodos conectados por la arista
            weight = grafo[node1][node2]["weight"]  # Obtener el peso de la arista
            if bitstring[node1] != bitstring[node2]:  # Comparar los bits
                cost += weight  # Aumentar el costo en el valor del peso de la arista si hay un corte

        if cost > best_cost:
            best_cost = cost
            best_bitstrings = [bitstring]
        elif cost == best_cost:
            best_bitstrings.append(bitstring)

    return best_cost, best_bitstrings

# Parte 6: Ejecución del algoritmo
def ejecutar_algoritmo(n_layers=1, N_limite=1):
    cost_optimo = None  # Inicializamos cost_optimo como None
    with open('Busqueda_exhaustiva.txt', 'w', encoding='utf-8') as BE:
    # Si el número de nodos es menor o igual al límite, hacemos la búsqueda exhaustiva
        if n_wires <= N_limite:
            Output_exhaustivo = exhaustive_search_maxcut(graph)
            cost_optimo = Output_exhaustivo[0]  # Asignamos el costo óptimo
            print("Búsqueda exhaustiva:")
            print("Costo óptimo (número de cortes óptimo):", cost_optimo)
            print("Mejores bitstrings:", Output_exhaustivo[1])
            BE.write(f"Costo Optimo {cost_optimo}\n")
            BE.write(f"Mejores Bitsrings: {Output_exhaustivo[1]}\n")
    # Ejecutar QAOA
    Output_qaoa = qaoa_maxcut(n_layers=n_layers)
    return Output_qaoa, cost_optimo  # Devolvemos ambos valores

# Ejecutar el algoritmo con un límite específico
N_limite = 15
n_layers=1
Output, cost_optimo = ejecutar_algoritmo(n_layers, N_limite=N_limite)

# Parte 7: Visualización de resultados


# 7.1 Generar histograma de bitstrings (solo si se ejecutó QAOA)
if Output[1] is not None:
    xticks = range(0, 2**n_wires)
    xtick_labels = list(map(lambda x: format(x, "0{}b".format(n_wires)), xticks))
    bins = np.arange(0, 2**n_wires + 1) - 0.5

    plt.figure(figsize=(10, 6))  # Aumentar el tamaño de la figura
    plt.title(f"(N_layers={n_layers})", fontsize=16)
    plt.xlabel("Bitstrings", fontsize=14)
    plt.ylabel("Frecuencia", fontsize=14)
    plt.xticks(xticks, xtick_labels, rotation="vertical", fontsize=10)  # Tamaño de las etiquetas
    plt.hist(Output[1], bins=bins, color='skyblue', edgecolor='black', alpha=0.7)  # Colores mejorados

    # Añadir una cuadrícula
    plt.grid(axis='y', alpha=0.75)

    # Guardar el histograma
    plt.tight_layout()
    plt.savefig(f'histograma_n_layers_{n_layers}.png')
    plt.show()

# 7.2 Costo vs número de iteraciones
plt.figure(figsize=(8, 6))
plt.title("Costo vs Número de iteraciones")
plt.xlabel("Número de iteraciones")
plt.ylabel("Costo")
plt.plot(range(len(Output[3])), Output[3], marker="o", label="Costo QAOA")

# Añadir línea del costo óptimo si está disponible
if cost_optimo is not None:
    plt.axhline(y=cost_optimo, color='r', linestyle='--', label="Costo Óptimo (Búsqueda Exhaustiva)")

plt.legend()
plt.tight_layout()

# Guardar el gráfico de costo
plt.savefig(f'costo_vs_iteraciones_n_layers_{n_layers}.png')
plt.show()

# 7.3 Dibuja el circuito
drawer = qml.draw(circuit)
circuito_cuatico = drawer(Output[2][0], Output[2][1], n_layers=n_layers)
print(circuito_cuatico)

# Guardar el circuito como un archivo de texto
with open('circuito_cuatico.txt', 'w', encoding='utf-8') as f:
    f.write(circuito_cuatico)